Voice Control
-

In [ ]:
import speech_recognition as sr
import tkinter as tk
import threading
from tkinter import ttk
from PIL import Image, ImageTk
import serial  # Import serial library for Arduino communication

## Voice recognition settings
recognizer = sr.Recognizer()
microphone = sr.Microphone()


## Attempt to connect to Arduino
try:
    arduino = serial.Serial('COM6', 9600, timeout=1)
except serial.SerialException:
    arduino = None
    print("⚠️ Could not connect to Arduino. Check the port and try again.")


## Multilingual Voice Command Dictionary
commands = {
    "english": {
        "forward": "F", 
        "backward": "B", "back": "B",
        "left": "L", "turn left": "L",
        "right": "R", "turn right": "R",
        "stop": "S", "stop moving": "S"
    },
    "arabic": {
        "للامام": "F", "للأمام": "F", "قدام": "F", "اطلع": "F", "أطلع": "F", "امام": "F", "أمام": "F",
        "للخلف": "B", "ورا": "B",
        "لليسار": "L", "شمال": "L", "يسار": "L",
        "لليمين": "R", "يمين": "R",
        "توقف": "S", "اوقف": "S", "أوقف": "S"
    },
    "french": {
        "avancer": "F", "en arrière": "B", "gauche": "L", "droite": "R", "arrêt": "S"
    },
    "spanish": {
        "adelante": "F", "atrás": "B", "izquierda": "L", "derecha": "R", "alto": "S"
    },
    "german": {
        "vorwärts": "F", "rückwärts": "B", "links": "L", "rechts": "R", "stopp": "S"
    },
    "italian": {
        "avanti": "F", "indietro": "B", "sinistra": "L", "destra": "R", "ferma": "S"
    },
    "turkish": {
        "ileri": "F", "geri": "B", "sol": "L", "sağ": "R", "dur": "S"
    },
    "hindi": {
        "आगे": "F", "पीछे": "B", "बाएं": "L", "दाएं": "R", "रुको": "S"
    },
    "chinese": {
        "前进": "F", "后退": "B", "左转": "L", "右转": "R", "停止": "S"
    },
    "japanese": {
        "前": "F", "後ろ": "B", "左": "L", "右": "R", "止まれ": "S"
    },
    "russian": {
        "вперед": "F", "назад": "B", "налево": "L", "направо": "R", "стоп": "S"
    },
    "korean": {
        "앞으로": "F", "뒤로": "B", "왼쪽": "L", "오른쪽": "R", "멈춰": "S"
    },
    "portuguese": {
        "frente": "F", "trás": "B", "esquerda": "L", "direita": "R", "parar": "S"
    },
    "polish": {
        "do przodu": "F", "do tyłu": "B", "w lewo": "L", "w prawo": "R", "stop": "S"
    },
    "vietnamese": {
        "tiến lên": "F", "lùi lại": "B", "trái": "L", "phải": "R", "dừng lại": "S"
    }
}

listening = False
paused = False
selected_language = "english"

def send_to_arduino(command):
    """Send recognized command to Arduino"""
    if arduino:
        arduino.write(command.encode())  # Convert to bytes and send
        print(f"📤 Sent to Arduino: {command}")
    else:
        print("⚠️ Arduino not connected!")

def recognize_speech():
    global listening, paused, selected_language
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        while listening:
            if paused:
                status_label.config(text="⏸️ Paused...")
                root.update()
                continue

            status_label.config(text="🎤 Listening...")
            root.update()

            try:
                language_codes = {
                    "english": "en", "arabic": "ar", "french": "fr",
                    "spanish": "es", "german": "de", "italian": "it",
                    "turkish": "tr", "hindi": "hi", "chinese": "zh-CN",
                    "japanese": "ja", "russian": "ru", "korean": "ko",
                    "portuguese": "pt", "polish": "pl", "vietnamese": "vi"
                }
                audio = recognizer.listen(source)
                text = recognizer.recognize_google(audio, language=language_codes[selected_language]).lower()
                result_label.config(text=f"🎧 Heard: {text}")

                if text in commands[selected_language]:
                    command = commands[selected_language][text]
                    command_label.config(text=f"📢 Command received: {command}")
                    send_to_arduino(command)  # Send command to Arduino
                else:
                    command_label.config(text="🚫 Unknown command.")

            except sr.UnknownValueError:
                result_label.config(text="❌ Could not understand.")
            except sr.RequestError:
                result_label.config(text="⚠️ Connection error with Google service.")

def start_listening():
    global listening, paused
    if not listening:
        listening = True
        paused = False
        threading.Thread(target=recognize_speech, daemon=True).start()
        status_label.config(text="🎤 Listening...")

def pause_listening():
    global paused
    if listening:
        paused = True
        status_label.config(text="⏸️ Paused...")

def resume_listening():
    global paused
    if listening:
        paused = False
        status_label.config(text="🎤 Listening...")

def change_language(event):
    global selected_language
    selected_language = language_var.get()
    language_names = {
        "english": "English", "arabic": "Arabic", "french": "French",
        "spanish": "Spanish", "german": "German", "italian": "Italian",
        "turkish": "Turkish", "hindi": "Hindi", "chinese": "Chinese",
        "japanese": "Japanese", "russian": "Russian", "korean": "Korean",
        "portuguese": "Portuguese", "polish": "Polish", "vietnamese": "Vietnamese"
    }
    lang_label.config(text=f"🌍 Selected Language: {language_names[selected_language]}")

# ======== GUI ========
root = tk.Tk()
root.title("Voice Command Recognition")
root.geometry("1920x1080")

## Load background image
bg_image = Image.open("Voice Control.jpg")
bg_image = bg_image.resize((1920, 1080))
bg_photo = ImageTk.PhotoImage(bg_image)

## Set background image
bg_label = tk.Label(root, image=bg_photo)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)

## Foreground elements on top of the background
button_frame = tk.Frame(root, bg="#87CEFA")
button_frame.pack(pady=20)

start_button = tk.Button(button_frame, text="▶️ Start", command=start_listening, font=("Arial", 14), bg="#228B22", fg="white", width=12, height=1)
start_button.pack(side=tk.LEFT, padx=10)

pause_button = tk.Button(button_frame, text="⏸️ Pause", command=pause_listening, font=("Arial", 14), bg="#1E90FF", fg="white", width=12, height=1)
pause_button.pack(side=tk.LEFT, padx=10)

resume_button = tk.Button(button_frame, text="▶️ Resume", command=resume_listening, font=("Arial", 14), bg="#FFA500", fg="white", width=12, height=1)
resume_button.pack(side=tk.LEFT, padx=10)

language_label = tk.Label(root, text="🌍 Select Language:", font=("Arial", 14), bg="#00BFFF")
language_label.pack()

language_var = tk.StringVar()
language_dropdown = ttk.Combobox(
    root, textvariable=language_var,
    values=[
        "english", "arabic", "french", "spanish", "german", "italian",
        "turkish", "hindi", "chinese", "japanese", "russian", "korean",
        "portuguese", "polish", "vietnamese"
    ],
    font=("Arial", 14), state="readonly"
)
language_dropdown.pack(pady=10)
language_dropdown.current(0)
language_dropdown.bind("<<ComboboxSelected>>", change_language)

lang_label = tk.Label(root, text="🌍 Selected Language: English", font=("Arial", 14, "bold"), bg="#00BFFF")
lang_label.pack(pady=5)

status_label = tk.Label(root, text="Press 'Start' to begin recognition", font=("Arial", 14), bg="#00BFFF")
status_label.pack(pady=10)

result_label = tk.Label(root, text="", font=("Arial", 14), bg="#00BFFF")
result_label.pack(pady=10)

command_label = tk.Label(root, text="", font=("Arial", 14, "bold"), bg="#00BFFF")
command_label.pack(pady=10)

root.mainloop()


In [ ]:
## Close the connection when the program ends
arduino.close()